# Caso de Estudio: Implementación de una Base de Datos Distribuida en Cassandra para la Plataforma de Pedidos "FastEat"

## Definición del problema

### Descripción del Escenario Empresarial 

"FastEat" es una cadena de restaurantes con presencia en varias ciudades de Latinoamérica. Para incrementar sus ventas y mejorar la experiencia de sus clientes, la empresa ha lanzado una plataforma digital de gestión de pedidos y entregas a domicilio, la cual permite a los usuarios realizar órdenes en línea, personalizar sus platos, rastrear el estado de sus pedidos en tiempo real y calificar el servicio recibido.

La expansión acelerada de la cadena y el éxito de la plataforma han hecho que el volumen de transacciones crezca rápidamente, especialmente en horarios pico y durante promociones especiales. Además, la empresa planea implementar un sistema de recomendaciones personalizadas para sugerir platillos a los clientes según su historial y ubicación, así como análisis en tiempo real de las preferencias de los usuarios.

La plataforma debe operar de manera continua (24/7), soportar miles de operaciones simultáneas y garantizar que la información de pedidos y entregas sea precisa, rápida y esté siempre disponible, incluso en caso de fallos o caídas de algún servidor.

### Problema a Resolver

El sistema de base de datos relacional utilizado inicialmente por FastEat ha comenzado a presentar graves problemas de rendimiento: demoras en la consulta de pedidos durante horas pico, dificultades para escalar la infraestructura ante el crecimiento de la demanda y limitaciones para integrar nuevos módulos como la recomendación personalizada o el análisis de datos en tiempo real.

Por este motivo, la dirección tecnológica de FastEat ha decidido migrar la plataforma a una base de datos distribuida basada en Cassandra, con el objetivo de asegurar escalabilidad, alta disponibilidad y tolerancia a fallos, además de facilitar el manejo eficiente de grandes volúmenes de datos y transacciones concurrentes.

### Objetivos del Proyecto

1. Analizar los requerimientos funcionales del sistema de pedidos y entregas de FastEat, identificando los tipos de datos principales, relaciones y operaciones críticas de consulta y escritura.
2. Diseñar un modelo de datos eficiente para Cassandra, aplicando las mejores prácticas de bases de datos NoSQL, optimizando consultas y garantizando la integridad de la información.
3. Definir la estructura de tablas, claves de partición y clustering keys más adecuadas para responder a los retos de la plataforma (consultas por usuario, seguimiento de pedidos, historial de entregas, etc.).
4. Proponer una estrategia de despliegue en contenedores Docker para la infraestructura de Cassandra, asegurando escalabilidad horizontal y alta disponibilidad. 
5. Justificar las decisiones de diseño en función de los escenarios de uso y los retos operativos descritos.
6. Los datos deben ser leidos (desde python con jupyther) de un archivo externo tipo json para que sean insertados en la tabla  
7. Desarrollar y ejecutar un conjunto de consultas esenciales sobre la base de datos implementada utilizando Python en Jupyter Notebook para la conexión y análisis de los datos. Las consultas deben demostrar la recuperación de pedidos, inserción de nuevos pedidos, actualización del estado de entregas y generación de reportes simples.

### Indicaciones para el Estudiante

Como desarrolladores responsables del proyecto, deberán analizar el escenario presentado y diseñar una solución de base de datos distribuida en Cassandra que cumpla con los requisitos de FastEat. Se espera un documento técnico donde describan el modelo de datos propuesto, justifiquen la elección de las tablas y claves, y detallen la estrategia de despliegue en Docker.

Además, deben preparar y ejecutar, en Jupyter Notebook, consultas relevantes sobre la base de datos usando Python (por ejemplo, empleando la librería `cassandra-driver`). Deberán incluir ejemplos de inserción de pedidos, consultas de estado, actualización de entregas y visualizaciones de datos relevantes.

El enfoque del trabajo debe estar en la eficiencia, escalabilidad y disponibilidad del sistema, demostrando dominio de la integración entre bases de datos distribuidas y herramientas modernas de análisis de datos.

## Docker setup

### Red

```bash
docker network create cassandra-net
```
### Nodos

#### `cassandra-1`

```bash
docker run -d --name cassandra-1 --memory=1.5g --network cassandra-net -e CASSANDRA_CLUSTER_NAME="fasteat-cluster" -e CASSANDRA_SEEDS="cassandra-1" -e CASSANDRA_BROADCAST_ADDRESS="cassandra-1" -p 9042:9042 cassandra
```

#### `cassandra-2`

```bash
docker run -d --name cassandra-2 --memory=2g --network cassandra-net -e CASSANDRA_CLUSTER_NAME="fasteat-cluster" -e CASSANDRA_SEEDS="cassandra-1" -e CASSANDRA_BROADCAST_ADDRESS="cassandra-2" cassandra
```

#### `cassandra-3`

```bash
docker run -d --name cassandra-3 --memory=2g --network cassandra-net -e CASSANDRA_CLUSTER_NAME="fasteat-cluster" -e CASSANDRA_SEEDS="cassandra-1" -e CASSANDRA_BROADCAST_ADDRESS="cassandra-3" cassandra
```

## Jupyter

In [1]:
# Importar las bibliotecas necesarias
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
from datetime import datetime

import uuid

### Posible JSON inicial

In [11]:
initial_data = [
    {
        'user': {
            'user_id': 'U00001',
            'name': 'Ana Torres',
            'phone': '0987654321',
            'city': 'Quito',
            'location': {
                'longitude': -78.4678,
                'latitude': -0.1807
            }
        },
        'orders': [
            {
                'order_id': 'O00001',
                'order_timestamp': '2025-07-16T10:12:00',
                'items': [
                    {
                        'item_id': 'I404',
                        'name': 'Pizza Margarita',
                        'quantity': 1,
                        'customizations': [],
                        'price': 10.00,
                        'total': 10.00
                    }
                ],
                'taxes': 1.50,
                'delivery_price': 1.80,
                'total': 13.30,
                'latest': {
                    'status': 'delivered',
                    'location': {
                        'longitude': -78.4678,
                        'latitude': -0.1807
                    },
                    'update_timestamp': '2025-07-16T10:50:00'
                },
                'review': {
                    'comments' : 'Me gustó el servicio!',
                    'rating': 5,
                    'timestamp': '2025-07-16T10:55:00'
                }
            }
        ]
    }
]

### Prueba de conexión

In [2]:
# Conectarse al cluster
cluster = Cluster(['localhost'], port=9042) # Cambiar 'localhost' por la IP del host si no se está ejecutando localmente
session = cluster.connect()

# Ejecutar una consulta simple
row = session.execute("SELECT release_version FROM system.local").one()
if row:
    print(f"Cassandra release version: {row[0]}")
else:
    print("An error occurred.")

Cassandra release version: 5.0.4


### Gestión de Keyspaces

In [3]:
# Listar los keyspaces existentes
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces")
for row in rows:
    print(row.keyspace_name)

system_auth
system_schema
system_distributed
system
system_traces


In [6]:
# Crear un keyspace
session.execute("CREATE KEYSPACE IF NOT EXISTS fast_eat WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '3'}")

In [5]:
# Eliminar el keyspace creado
session.execute("DROP KEYSPACE IF EXISTS fast_eat")

In [7]:
# Usar el keyspace 'fast_eat'
session.set_keyspace('fast_eat')

In [8]:
# Crear tipo de dato location
session.execute("""
    CREATE TYPE IF NOT EXISTS location (
        longitude FLOAT,
        latitude FLOAT
    )
""")

### Gestión de Tablas

In [13]:
# Crear tabla de usuarios
session.execute("""
    CREATE TABLE IF NOT EXISTS users (
        user_id TEXT PRIMARY KEY,
        name TEXT,
        phone TEXT,
        city TEXT,
        location FROZEN<location>
    )
""")

In [14]:
class Location:
    def __init__(self, longitude, latitude):
        self.longitude = longitude
        self.latitude = latitude

cluster.register_user_type('fast_eat', 'location', Location)

# Insertar datos iniciales
for data in initial_data:
    user = data['user']
    location = Location(user['location']['longitude'], user['location']['latitude'])
    session.execute("""
        INSERT INTO users (user_id, name, phone, city, location)
        VALUES (%s, %s, %s, %s, %s)
    """, (user['user_id'], user['name'], user['phone'], user['city'], location))

In [15]:
# Mostrar los usuarios insertados
rows = session.execute("SELECT * FROM users")
for row in rows:
    print(f"User ID: {row.user_id}, Name: {row.name}, Phone: {row.phone}, City: {row.city}, Location: ({row.location.longitude}, {row.location.latitude})")

User ID: U00001, Name: Ana Torres, Phone: 0987654321, City: Quito, Location: (-78.4677963256836, -0.18070000410079956)


In [ ]:
# Crear tabla de pedidos por usuario en Cassandra
session.execute("""
    CREATE TABLE IF NOT EXISTS orders_by_user (
        user_id TEXT,
        order_id TEXT,
        order_date TIMESTAMP,
        items LIST<TEXT>,
        total FLOAT,
        status TEXT,
        PRIMARY KEY (user_id, order_date, order_id)
    )
""")